<a href="https://colab.research.google.com/github/YaninaK/sales-forecast/blob/main/notebooks/04_Transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sales forecast for 20 stores for 31 days' period.

## Transformer model

[EDA: omissions, clustering & Fourier transformations](
https://colab.research.google.com/drive/1udCgyEt7lusSY43lnduVDEnZvdvVEt9h?usp=sharing)

[Feature extraction](https://colab.research.google.com/drive/1H-vQoHnycp62YuZD1xrmIDjufpDu7ig3?usp=sharing)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone -q https://github.com/YaninaK/sales-forecast.git
!pip install -r sales-forecast/requirements_Colab.txt -q 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.2/358.2 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 KB 14.9 MB/s eta 0:00:00


In [3]:
%cd sales-forecast

/content/sales-forecast


In [4]:
import sys
import os

sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "src", "sales_forecast"))

In [5]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib

from data.make_dataset import load_data
from src.sales_forecast.models import train

from data.johnson_su_transformation import JohnsonSU
from data.train_test_datasets import get_train_dataset, get_test_dataset
from models.model_TF import get_model_TF

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
PATH = '/content/drive/MyDrive/ML_projects/01_Time_series/sales_forecast/'
DEFAULT_RANDOM_SEED = 3

In [8]:
def set_all_seeds(seed=DEFAULT_RANDOM_SEED):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  
set_all_seeds()

## 1. Data ingestion

In [9]:
data = load_data(PATH)

print(f'data.shape = {data.shape}\n')
data.head(2)

data.shape = (8230, 3)



,id,dt,target
0,0,2016-01-02,747.8
1,0,2016-01-03,681.2


## 2. Data preprocessing pipeline

In [10]:
regenerate = True
if regenerate:
  X = train.data_preprocessing_pipeline(data)
else:
  X = np.load('data/04_feature/train_dataset.npy')

X.shape

Number of outliers = 8


(153, 20, 40)

## 3. Transformer model

### 3.1 Train dataset

In [11]:
X_train, y_train = get_train_dataset(X)
X_train.shape, y_train.shape

((900, 54, 40), (900, 27, 40))

### 3.2 Training of the model

In [12]:
model = get_model_TF(X_train.shape[1:])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 54, 40)]     0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 54, 40)      80          ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 transformer_block (Transformer  (None, 54, 40)      52432       ['layer_normalization[0][0]']    
 Block)                                                                                           
                                                                                              

In [13]:
n_shops = 20
n_epochs = 25
batch_size = n_shops * 2
m = n_shops * 4

reduce_lr = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 3e-2 * 0.95 ** epoch
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.Huber()
)
history = model.fit(
    X_train[:-m, :, :], y_train[:-m, :, :], 
    epochs=n_epochs,
    validation_data=(X_train[-m:, :, :], y_train[-m:, :, :]),
    batch_size=batch_size, 
    verbose=1,
    callbacks=[reduce_lr],
    shuffle=True,
    workers=-1,
    use_multiprocessing=True
)

Epoch 1/25
21/21 [==============================] - 19s 281ms/step - loss: 0.4455 - val_loss: 420.5376 - lr: 0.0300
Epoch 2/25
21/21 [==============================] - 4s 209ms/step - loss: 0.3856 - val_loss: 3.2996 - lr: 0.0285
Epoch 3/25
21/21 [==============================] - 5s 261ms/step - loss: 0.3756 - val_loss: 0.8346 - lr: 0.0271
Epoch 4/25
21/21 [==============================] - 5s 223ms/step - loss: 0.3674 - val_loss: 1.0716 - lr: 0.0257
Epoch 5/25
21/21 [==============================] - 5s 220ms/step - loss: 0.3580 - val_loss: 0.6974 - lr: 0.0244
Epoch 6/25
21/21 [==============================] - 6s 274ms/step - loss: 0.3461 - val_loss: 0.4983 - lr: 0.0232
Epoch 7/25
21/21 [==============================] - 4s 214ms/step - loss: 0.3380 - val_loss: 0.5034 - lr: 0.0221
Epoch 8/25
21/21 [==============================] - 5s 248ms/step - loss: 0.3305 - val_loss: 0.4883 - lr: 0.0210
Epoch 9/25
21/21 [==============================] - 5s 210ms/step - loss: 0.3234 - val_loss: 

### 3.3 Inference

In [14]:
X_test, y_test = get_test_dataset(X)

In [15]:
pred = model.predict(X_test)
pred.shape

1/1 [==============================] - 1s 1s/step


(20, 27, 40)

### 3.4 Model evaluation

#### Mean absolute scaled error (MASE) 
https://en.wikipedia.org/wiki/Mean_absolute_scaled_error

The denominator of mean absolute scaled error is the mean absolute error of the one-step "seasonal naive forecast method" on the training set, which uses the actual value from the prior 30 days as the forecast: 
* F(t) = Y(t−30)

MASE is scale-free error metric. So, it can be applied to scaled data without extra iteration of inverse scaling.

In [16]:
pred.shape, y_test.shape

((20, 27, 40), (20, 27, 40))

In [17]:
forecast_error = abs(pred[:, :, 0] - y_test[:, :, 0]).mean()
mase = forecast_error / abs(y_test[:, :, 6]).mean()

print(f'Mean absolute scaled error = {round(mase.mean(), 4)}')

Mean absolute scaled error = 0.9395


__Findings:__

* Out of 72 samples available for training (153 less input and output sequence length) 4 were used for validation of the model and 27 for testing. So, we had only 41 samples for training of the model and it is definately not sufficient.

* To get more accurate predictions, we need to retrain the model on the full set of data. It should improve the final results.

## 4. Final prediction

### 4.1. Dataset

In [18]:
X_train, y_train = get_train_dataset(X, final=True)
X_test = get_test_dataset(X, final=True)

X_train.shape, y_train.shape, X_test.shape

((1440, 54, 40), (1440, 27, 40), (20, 54, 40))

### 4.2. Retraining the model on full dataset

In [19]:
n_shops = 20
n_epochs = 30
batch_size = n_shops * 2
m = n_shops * 4

model = model = get_model_TF(X_train.shape[1:])
reduce_lr = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 3e-2 * 0.95 ** epoch
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss=tf.keras.losses.Huber()
)
history = model.fit(
    X_train[:-m, :, :], y_train[:-m, :, :], 
    epochs=n_epochs,
    validation_data=(X_train[-m:, :, :], y_train[-m:, :, :]),
    batch_size=batch_size, 
    verbose=1,
    callbacks=[reduce_lr],
    shuffle=True,
    workers=-1,
    use_multiprocessing=True,    
)

Epoch 1/30
34/34 [==============================] - 19s 223ms/step - loss: 0.3955 - val_loss: 121.8461 - lr: 0.0300
Epoch 2/30
34/34 [==============================] - 8s 238ms/step - loss: 0.3509 - val_loss: 1.2153 - lr: 0.0285
Epoch 3/30
34/34 [==============================] - 7s 203ms/step - loss: 0.3428 - val_loss: 0.4403 - lr: 0.0271
Epoch 4/30
34/34 [==============================] - 8s 235ms/step - loss: 0.3387 - val_loss: 0.3709 - lr: 0.0257
Epoch 5/30
34/34 [==============================] - 8s 227ms/step - loss: 0.3346 - val_loss: 0.3589 - lr: 0.0244
Epoch 6/30
34/34 [==============================] - 7s 204ms/step - loss: 0.3299 - val_loss: 0.3513 - lr: 0.0232
Epoch 7/30
34/34 [==============================] - 8s 237ms/step - loss: 0.3252 - val_loss: 0.3712 - lr: 0.0221
Epoch 8/30
34/34 [==============================] - 7s 202ms/step - loss: 0.3193 - val_loss: 0.3779 - lr: 0.0210
Epoch 9/30
34/34 [==============================] - 8s 235ms/step - loss: 0.3112 - val_loss: 

### 4.3 Model evaluation on validation set

In [20]:
pred = model.predict(X_train[-m:, :, :])
pred[:, :, 0].shape,

3/3 [==============================] - 1s 48ms/step


((80, 27),)

In [21]:
forecast_error = abs(pred[:, :, 0] -  y_train[-m:, :, 0]).mean()
mase = forecast_error / abs( y_train[-m:, :, 6]).mean()

print(f'Mean absolute scaled error = {round(mase.mean(), 4)}')

Mean absolute scaled error = 0.5538


### 4.4. Inference

In [22]:
pred_final = model.predict(X_test)

1/1 [==============================] - 0s 59ms/step


### 4.5. Inverse transformation of predictions

In [23]:
scaler = joblib.load('data/03_primary/scaler_js.joblib')

In [24]:
arr_pred = pred_final[:, :, 0]
y_pred = pd.DataFrame(arr_pred).T
y_pred_rescaled = scaler.inv_transform(y_pred)
y_pred_rescaled.shape
     

(27, 20)

### 4.6. Generation of the final result

In [25]:
predict_example = pd.read_parquet(PATH + 'data/01_raw/sample_submit.parquet.gzip')
print(f'test.shape = {predict_example.shape}\n')
predict_example.head(2)

test.shape = (540, 3)



,id,dt,target
0,0,2017-07-01,500
1,0,2017-07-02,500


In [26]:
n_shops = 20
for i in range(n_shops):
  predict_example.loc[predict_example['id'] == i, 'target'] = y_pred_rescaled.loc[:, i]

predict_example.to_parquet(
    'data/06_model_output/sales_forecast_tf.parquet.gzip', compression='gzip'
)
predict_example.head()

,id,dt,target
0,0,2017-07-01,645.115981
1,0,2017-07-02,654.070780
2,0,2017-07-03,587.311412
3,0,2017-07-04,581.064176
4,0,2017-07-06,654.167037


## 5. Findings

Transformer model has not outperformed LSTM model. There is not enough data for transformers. The use of transformers could be justified on the longer time sequences.